In [31]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import yfinance as yf
import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup
import nltk
from tqdm import tqdm

from finBERT_functions import *

In [37]:
ticker_news = yf.Ticker("GOOG").news

dict_list = []

labels = {0:'neutral', 1:'positive', 2:'negative'}

for news in tqdm(ticker_news):
    timestamp = news['providerPublishTime']
    datetime = dt.datetime.fromtimestamp(timestamp)

    url = news['link']
    response = requests.get(url)

    if response.status_code != 200:
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    

    article = soup.find('div', class_='caas-body').text

    # tokenize the article as sentences
    sentences = nltk.sent_tokenize(article)

    news_dict = {
        'date': datetime,
        'title': news['title'],
        'link': news['link'],
    }

    news_dict.update(get_sentiment(sentences))

    dict_list.append(news_dict)

    print(news_dict)

df = pd.DataFrame(dict_list, index=[x['date'] for x in dict_list]).drop(columns=['date'])

display(df)

 12%|█▎        | 1/8 [00:03<00:21,  3.08s/it]

{'date': datetime.datetime(2023, 11, 1, 16, 8, 19), 'title': "Biden's AI order is massive, but far from enough to address the technology's risks", 'link': 'https://finance.yahoo.com/news/bidens-ai-order-is-massive-but-far-from-enough-to-address-the-technologys-risks-150819906.html', 0: 0.7619047619047619, 1: 0.14285714285714285, 2: 0.09523809523809523}


 25%|██▌       | 2/8 [00:03<00:08,  1.46s/it]

{'date': datetime.datetime(2023, 11, 1, 15, 45), 'title': '2 Media Stocks Rivaling Netflix to Buy Hand Over Fist', 'link': 'https://finance.yahoo.com/m/a37b15d3-c29e-3e54-846b-9cf2634045d1/2-media-stocks-rivaling.html', 1: 1.0}


 50%|█████     | 4/8 [00:09<00:11,  2.89s/it]

{'date': datetime.datetime(2023, 11, 1, 14, 21, 40), 'title': "The 'Magnificent Seven' stock market leaders went their separate ways in October", 'link': 'https://finance.yahoo.com/news/the-magnificent-seven-stock-market-leaders-went-their-separate-ways-in-october-132140735.html', 2: 0.5, 1: 0.42857142857142855, 0: 0.07142857142857142}


100%|██████████| 8/8 [00:16<00:00,  2.00s/it]

{'date': datetime.datetime(2023, 11, 1, 12, 0, 17), 'title': 'Former Google CEO Eric Schmidt Bets AI Will Shake Up\xa0Scientific Research', 'link': 'https://finance.yahoo.com/news/former-google-ceo-eric-schmidt-110017770.html', 0: 1.0}


,title,link,0,1,2
2023-11-01 16:08:19,"Biden's AI order is massive, but far from enou...",https://finance.yahoo.com/news/bidens-ai-order...,0.761905,0.142857,0.095238
2023-11-01 15:45:00,2 Media Stocks Rivaling Netflix to Buy Hand Ov...,https://finance.yahoo.com/m/a37b15d3-c29e-3e54...,NaN,1.000000,NaN
2023-11-01 14:21:40,The 'Magnificent Seven' stock market leaders w...,https://finance.yahoo.com/news/the-magnificent...,0.071429,0.428571,0.500000
2023-11-01 12:00:17,Former Google CEO Eric Schmidt Bets AI Will Sh...,https://finance.yahoo.com/news/former-google-c...,1.000000,NaN,NaN
